# Assignment 6

Maksim Nikiforov

## Split the data

Using the NFL box score data set, split the data into separate .csv files based on the season.

In [1]:
import pandas as pd

# Read in our .csv file
NFL_scores = pd.read_csv("scoresFull.csv")

# Show the first set of rows
NFL_scores.head()

,week,date,day,season,awayTeam,AQ1,AQ2,AQ3,AQ4,AOT,...,homeFumLost,homeNumPen,homePenYds,home3rdConv,home3rdAtt,home4thConv,home4thAtt,homeTOP,HminusAScore,homeSpread
0,1,5-Sep,Thu,2002,San Francisco 49ers,3,0,7,6,-1,...,0,10,80,4,8,0,1,32.47,-3,-4.0
1,1,8-Sep,Sun,2002,Minnesota Vikings,3,17,0,3,-1,...,1,4,33,2,6,0,0,28.48,4,4.5
2,1,8-Sep,Sun,2002,New Orleans Saints,6,7,7,0,6,...,0,8,85,1,6,0,1,31.48,-6,6.0
3,1,8-Sep,Sun,2002,New York Jets,0,17,3,11,6,...,1,10,82,4,8,2,2,39.13,-6,-3.0
4,1,8-Sep,Sun,2002,Arizona Cardinals,10,3,3,7,-1,...,0,7,56,6,10,1,2,34.40,8,6.0


In [2]:
# Group data frame by each season with groupby,
# then write to .csv using .to_csv() as covered in module 3
for (season), group in NFL_scores.groupby(['season']):
     group.to_csv(f'season_{season}.csv', index=False)

## MapReduce

In [3]:
# Read in and store each output data set as an element of a list (season_data_list, which is iterable)
season_data_list = []
for year in range(2002, 2015):
    with open('season_' + str(year) + '.csv', 'r') as f:
        season_data_list.append(f.read())

In [4]:
sample_data = pd.read_csv("season_2002.csv")

In [20]:
sample_data["AQ1_squared"] = sample_data.AQ1.pow(2)
sample_data

,week,date,day,season,awayTeam,AQ1,AQ2,AQ3,AQ4,AOT,...,homeNumPen,homePenYds,home3rdConv,home3rdAtt,home4thConv,home4thAtt,homeTOP,HminusAScore,homeSpread,AQ1_squared
0,1,5-Sep,Thu,2002,San Francisco 49ers,3,0,7,6,-1,...,10,80,4,8,0,1,32.47,-3,-4.0,9
1,1,8-Sep,Sun,2002,Minnesota Vikings,3,17,0,3,-1,...,4,33,2,6,0,0,28.48,4,4.5,9
2,1,8-Sep,Sun,2002,New Orleans Saints,6,7,7,0,6,...,8,85,1,6,0,1,31.48,-6,6.0,36
3,1,8-Sep,Sun,2002,New York Jets,0,17,3,11,6,...,10,82,4,8,2,2,39.13,-6,-3.0,0
4,1,8-Sep,Sun,2002,Arizona Cardinals,10,3,3,7,-1,...,7,56,6,10,1,2,34.40,8,6.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,Division,12-Jan,Sun,2002,New York Jets,3,7,0,0,-1,...,8,70,2,6,1,1,30.93,20,5.5,9
263,Division,12-Jan,Sun,2002,San Francisco 49ers,3,3,0,0,-1,...,10,100,8,10,0,0,36.77,25,6.0,9
264,ConfChamp,19-Jan,Sun,2002,Tennessee Titans,7,10,7,0,-1,...,14,127,1,4,0,0,26.42,17,9.0,49
265,ConfChamp,19-Jan,Sun,2002,Tampa Bay Buccaneers,10,7,3,7,-1,...,5,45,2,7,1,2,29.12,-17,4.0,100


In [59]:
grouped_data = sample_data[["week", "AQ1", "AQ1_squared"]]
grouped_data.groupby("week").agg({"AQ1": ["sum", "count"], "AQ1_squared": ["sum"]})

AQ1       AQ1_squared
          sum count         sum
week                           
1          81    16         695
10         78    14         686
11         52    16         450
12         68    16         666
13         56    16         534
14         88    16         908
15         50    16         388
16         92    16         906
17         51    16         419
2          67    16         695
3          44    14         314
4          69    14         459
5          30    14         272
6          70    14         708
7          23    14         125
8          34    14         312
9          71    14         661
ConfChamp  17     2         149
Division    6     4          18
SuperBowl   3     1           9
WildCard   28     4         294

In [79]:
# Write a mapping function. 
# Take in one of the data sets (one season of data), a grouping variable, and a target variable.
def map_stats(data, groupvar, targetvar):
    # Square the values of the target variable and store the result in a new column
    data[str(targetvar) + "_squared"] = data[targetvar].pow(2)
    # Subset the data frame to only include the grouping column, the target column, 
    # and the new column with squared values
    squared_targetvar = data[[groupvar, targetvar, str(targetvar) + "_squared"]]
    # Group the data using the grouping variable and display the sum and count for each column.
    # reset_index() to ensure that "week" ends up as a column
    grouped_data = squared_targetvar.groupby(groupvar).agg({targetvar: ["sum", "count"], str(targetvar) + "_squared": ["sum"]}).reset_index()
    # Convert to a dictionary of key-value pairs and return the result, referencing instructions from
    # https://stackoverflow.com/questions/26716616/convert-a-pandas-dataframe-to-a-dictionary
    # Lists are ordered as [sum_of_targetvar, count, sum_of_squared_targetvar]
    grouped_data_list = grouped_data.set_index("week").T.to_dict("list")
    return grouped_data_list

In [82]:
# Confirm that the mapping function works using data from
# the season_2002.csv.
# Recall that lists (values) are ordered as [sum_of_targetvar, count, sum_of_squared_targetvar]
map_stats(sample_data, "week", "AQ1")

{'1': [81, 16, 695],
 '10': [78, 14, 686],
 '11': [52, 16, 450],
 '12': [68, 16, 666],
 '13': [56, 16, 534],
 '14': [88, 16, 908],
 '15': [50, 16, 388],
 '16': [92, 16, 906],
 '17': [51, 16, 419],
 '2': [67, 16, 695],
 '3': [44, 14, 314],
 '4': [69, 14, 459],
 '5': [30, 14, 272],
 '6': [70, 14, 708],
 '7': [23, 14, 125],
 '8': [34, 14, 312],
 '9': [71, 14, 661],
 'ConfChamp': [17, 2, 149],
 'Division': [6, 4, 18],
 'SuperBowl': [3, 1, 9],
 'WildCard': [28, 4, 294]}